In [60]:
%matplotlib inline

In [61]:
import requests
import re
from fastai.text import *
import torch

In [62]:
book = requests.get('https://www.gutenberg.org/files/36/36-0.txt').text

In [63]:
start_iter = re.finditer('BOOK ONE', book)

In [64]:
next(start_iter)

<_sre.SRE_Match object; span=(1067, 1075), match='BOOK ONE'>

Commenting out some print lines here to shrink the notebook -- I did some exploration to figure out where the actual story began and ended (excluding table of contents, acknowledgements, etc.)

In [65]:
# book[1067:]

In [66]:
next(start_iter)

<_sre.SRE_Match object; span=(1920, 1928), match='BOOK ONE'>

In [67]:
# book[1920:]

In [68]:
book = book[1920:]

In [69]:
end_iter = re.finditer("among the dead.", book)

In [70]:
next(end_iter)

<_sre.SRE_Match object; span=(64109, 64124), match='among the dead '>

In [71]:
# book[:64125]

In [72]:
next(end_iter)

<_sre.SRE_Match object; span=(345594, 345609), match='among the dead.'>

In [73]:
# book[:345609]

In [74]:
book = book[:345609]

In [75]:
book = re.sub(r'[\r\n]+', ' ', str(book))

In [76]:
# book

In [77]:
tokens = [i.replace(',', '') for i in book.split('.')]

In [78]:
# tokens

In [79]:
tokens[:10]

['BOOK ONE THE COMING OF THE MARTIANS I',
 ' THE EVE OF THE WAR',
 ' No one would have believed in the last years of the nineteenth century that this world was being watched keenly and closely by intelligences greater than manâ\x80\x99s and yet as mortal as his own; that as men busied themselves about their various concerns they were scrutinised and studied perhaps almost as narrowly as a man with a microscope might scrutinise the transient creatures that swarm and multiply in a drop of water',
 ' With infinite complacency men went to and fro over this globe about their little affairs serene in their assurance of their empire over matter',
 ' It is possible that the infusoria under the microscope do the same',
 ' No one gave a thought to the older worlds of space as sources of human danger or thought of them only to dismiss the idea of life upon them as impossible or improbable',
 ' It is curious to recall some of the mental habits of those departed days',
 ' At most terrestrial men fa

In [80]:
! rm book_text.csv

In [81]:
with open('book_text.csv', 'a+') as file:
    for sentence in tokens:
        file.write(sentence + '\n')

In [82]:
with open('book_text.csv') as file:
    lines = file.readlines()[:5]
print(lines[0])
print(lines[1])
print(lines[2])

BOOK ONE THE COMING OF THE MARTIANS I

 THE EVE OF THE WAR

 No one would have believed in the last years of the nineteenth century that this world was being watched keenly and closely by intelligences greater than manâs and yet as mortal as his own; that as men busied themselves about their various concerns they were scrutinised and studied perhaps almost as narrowly as a man with a microscope might scrutinise the transient creatures that swarm and multiply in a drop of water



In [83]:
pd.read_csv('book_text.csv').iloc[20][0]

' And looking across space with instruments and intelligences such as we have scarcely dreamed of they see at its nearest distance only 35000000 of miles sunward of them a morning star of hope our own warmer planet green with vegetation and grey with water with a cloudy atmosphere eloquent of fertility with glimpses through its drifting cloud wisps of broad stretches of populous country and narrow navy-crowded seas'

In [84]:
data = TextLMDataBunch.from_csv('.', 'book_text.csv', text_cols=0)

In [85]:
data

TextLMDataBunch;

Train: LabelList (2434 items)
x: LMTextList
xxbos i went into my garden before breakfast and stood listening but towards the common there was nothing stirring but a xxunk,xxbos xxmaj some of those who saw its flight say it travelled with a hissing sound,xxbos xxmaj then by the light of their own destruction i saw them staggering and falling and their xxunk turning to run,xxbos xxmaj there were already a couple of score of passengers aboard some of whom had xxunk their last money in securing a passage but the captain lay off the xxmaj xxunk until five in the afternoon picking up passengers until the xxunk decks were even xxunk crowded,xxbos xxmaj flying hussars have been galloping into xxmaj chertsey
y: LMLabelList
,,,,
Path: .;

Valid: LabelList (609 items)
x: LMTextList
xxbos xxmaj it may be in my time xxunk men,xxbos xxmaj he was as black as a xxunk with the black dust alive but helplessly and xxunk drunk,xxbos xxmaj as the green flames lifted i could see the xxunk 

In [86]:
data.train_ds[0]

(Text xxbos i went into my garden before breakfast and stood listening but towards the common there was nothing stirring but a xxunk,
 EmptyLabel )

In [87]:
data.train_ds[1][0].data

array([   2,    5,   70,   11,  144,  112,   67,   59,  469,  517,   18, 2019,   21,   12,  669,  228])

In [88]:
data.valid_ds[1][0].data

array([   2,    5,   24,   16,   22,   73,   22,   12,    0,   21,    9,   73,  358,  672,   31, 1557,   10,    0,
       2112])

In [89]:
data.one_batch()[0].shape

torch.Size([64, 70])

In [90]:
x, y = data.one_batch()

In [91]:
x.shape, y.shape

(torch.Size([64, 70]), torch.Size([64, 70]))

In [92]:
x[0], y[0]

(tensor([ 159, 2023,   11,  520,  864,  194,   27,    9,  195,  196,   14,    5,
         2024,    5, 1518,  197,  203,    9,    5,  671,   29,    5,  571,  754,
          430,   26,    0,   55,  572,   11,  221,   68,    9,    0,   46,    5,
            0,    2,    5,   48,  296,   13,  396,   12,  167,  166,   12,  573,
         1519,   29, 1219,    0, 1018, 1520,  101,    9,   74, 2025,    2,    5,
            9,  865,  755,   26,  431,  672,   14,    9,    0,   11]),
 tensor([2023,   11,  520,  864,  194,   27,    9,  195,  196,   14,    5, 2024,
            5, 1518,  197,  203,    9,    5,  671,   29,    5,  571,  754,  430,
           26,    0,   55,  572,   11,  221,   68,    9,    0,   46,    5,    0,
            2,    5,   48,  296,   13,  396,   12,  167,  166,   12,  573, 1519,
           29, 1219,    0, 1018, 1520,  101,    9,   74, 2025,    2,    5,    9,
          865,  755,   26,  431,  672,   14,    9,    0,   11,    9]))

In [93]:
len(data.vocab.itos)

3096

In [94]:
nv = len(data.vocab.itos)

In [95]:
nh = 64

In [271]:
bs = 63

In [272]:
nv, nh, bs

(3096, 64, 63)

In [300]:
class BasicLanguageModel(nn.Module):
     
    def __init__(self):
        super().__init__()
        self.i_h = nn.Embedding(nv, nh)
        self.h_h = nn.Linear(nh, nh)
        self.h_o = nn.Linear(nh, nv)
        self.bn = nn.BatchNorm1d(nh)
        self.reset()
        
    def forward(self, x):
        res = []
        h = self.h
#         print(h.size())
        for i in range(x.shape[1]):
#             print(self.i_h(x[:,i]).size())
            h = h + self.i_h(x[:,i]) 
            h = F.relu(self.h_h(h))
            res.append(self.bn(h))
        self.h = h.detach()
        res = torch.stack(res, dim=1)
        return self.h_o(res)
    
    def reset(self):
        self.h = torch.zeros(1, nh).cuda()

In [334]:
class BasicLanguageModel2(nn.Module):
     
    def __init__(self, bs):
        super().__init__()
        self.i_h = nn.Embedding(nv, nh)
        self.h_h = nn.Linear(nh, nh)
        self.h_o = nn.Linear(nh, nv)
        self.bn = nn.BatchNorm1d(nh)
        self.bs = bs
        self.reset()
        
    def forward(self, x):
        res = []
        h = self.h
        for i in range(x.shape[1]):
            h = h + self.i_h(x[bs,i]) 
            h = F.relu(self.h_h(h))
            res.append(self.bn(h))
        print(len(res))
        print(res[0].size())
        self.h = h.detach()
        res = torch.stack(res, dim=1)
        return self.h_o(res)
    
    def reset(self):
        self.h = torch.zeros(self.bs, nh).cuda()

In [335]:
learn = Learner(data, BasicLanguageModel2(64), metrics=accuracy)

In [336]:
learn.summary()

IndexError: index 63 is out of bounds for dimension 0 with size 1

In [307]:
data.show_batch()

idx,text
0,it simply as a thing done a memory infinitely disagreeable but quite without the quality of remorse xxbos xxmaj xxunk i know xxbos a reddish flash at the edge the slightest projection of the outline just as the xxunk struck midnight ; and at that i told xxmaj ogilvy and he took my place xxbos xxmaj many of them like my brother seemed inclined to xxunk in the place xxbos
1,one perhaps two hundred yards from me the other towards xxmaj xxunk xxbos i went down xxunk the door and let him in and locked the door again xxbos xxmaj in the bar i afterwards found some biscuits and xxunk that had been overlooked xxbos xxmaj he appeared to be standing and yelling for no reason that i could xxunk xxbos xxunk you see a man in the xxunk i
2,made their footing good and crippled the greatest power in the world xxbos xxmaj there were raised voices and some sort of struggle appeared to be going on about the pit xxbos xxmaj we could do nothing but wait in aching xxunk during those two weary days xxbos xxmaj even the birds were xxunk and as we hurried along i and the artilleryman talked in whispers and looked now and
3,xxbos xxmaj xxunk with heliographs were pushing forward upon them from all sides xxbos xxmaj it was a xxunk xxunk of two notes âulla ulla ulla ullaâ keeping on perpetually xxbos xxmaj for it was xxunk the xxmaj martians were about us xxbos i crouched at the foot of the staircase with my back to the wall shivering violently xxbos xxmaj henderson went into the railway station at once in
4,xxunk âem away xxbos xxmaj able - xxunk clean - xxunk women we want xxunk and xxunk xxbos xxmaj xxunk xxunk my brother saw the large crescent of shipping already writhing with the approaching terror ; one ship passing behind another another coming round from xxunk to end on xxunk xxunk and giving off volumes of steam xxunk being let out xxunk rushing xxunk and thither xxbos xxmaj all the


In [308]:
learn.data.bs

64

In [309]:
learn.fit_one_cycle(10, max_lr=3e-2)

epoch,train_loss,valid_loss,accuracy,time
0,3.376422,5.044417,0.203962,00:01
1,3.384320,5.251276,0.198214,00:00
2,3.425472,5.417654,0.184989,00:01
3,3.443935,5.411005,0.189174,00:01
4,3.409564,5.488659,0.192467,00:00
5,3.334642,5.610497,0.193806,00:00
6,3.231488,5.688180,0.193359,00:00
7,3.117336,5.793586,0.192522,00:00
8,3.007167,5.835362,0.192020,00:00
9,2.917154,5.835832,0.191462,00:01


In [310]:
learn.predict("Hello ")

TypeError: only integer tensors of a single element can be converted to an index

In [311]:
learn.summary()

BasicLanguageModel
Layer (type)         Output Shape         Param #    Trainable 
Embedding            [64]                 198,144    True      
______________________________________________________________________
Linear               [64]                 4,160      True      
______________________________________________________________________
Linear               [70, 3096]           201,240    True      
______________________________________________________________________
BatchNorm1d          [64]                 128        True      
______________________________________________________________________

Total params: 403,672
Total trainable params: 403,672
Total non-trainable params: 0
Optimized with 'torch.optim.adam.Adam', betas=(0.9, 0.99)
Using true weight decay as discussed in https://www.fast.ai/2018/07/02/adam-weight-decay/ 
Loss function : FlattenedLoss
Callbacks functions applied 

In [312]:
p = LanguageLearner(data, BasicLanguageModel(), metrics=accuracy)

In [313]:
learn.save("basic_model")

In [314]:
p.load("basic_model")

LanguageLearner(data=TextLMDataBunch;

Train: LabelList (2434 items)
x: LMTextList
xxbos i went into my garden before breakfast and stood listening but towards the common there was nothing stirring but a xxunk,xxbos xxmaj some of those who saw its flight say it travelled with a hissing sound,xxbos xxmaj then by the light of their own destruction i saw them staggering and falling and their xxunk turning to run,xxbos xxmaj there were already a couple of score of passengers aboard some of whom had xxunk their last money in securing a passage but the captain lay off the xxmaj xxunk until five in the afternoon picking up passengers until the xxunk decks were even xxunk crowded,xxbos xxmaj flying hussars have been galloping into xxmaj chertsey
y: LMLabelList
,,,,
Path: .;

Valid: LabelList (609 items)
x: LMTextList
xxbos xxmaj it may be in my time xxunk men,xxbos xxmaj he was as black as a xxunk with the black dust alive but helplessly and xxunk drunk,xxbos xxmaj as the green flames lifted i

In [317]:
p.predict(torch.tensor([301]), n_words=10)

IndexError: index 1 is out of bounds for dimension 0 with size 1

In [232]:
torch.save(mdl.state_dict(), "./MDL_DICT")

In [270]:
mdl2 = BasicLanguageModel2().eval().cuda()
mdl2.load_state_dict(torch.load("./MDL_DICT"))

RuntimeError: Error(s) in loading state_dict for BasicLanguageModel2:
	Unexpected key(s) in state_dict: "bn.weight", "bn.bias", "bn.running_mean", "bn.running_var", "bn.num_batches_tracked". 

In [252]:
mdl = l.eval()

NameError: name 'l' is not defined

In [253]:
mdl(torch.tensor(0).view(-1, 1, 1).cuda()).size()

torch.Size([1, 1, 64, 64])


torch.Size([1, 1, 64, 3096])

In [191]:
idxs = [data.train_ds.vocab.stoi[w] for w in "The world is".split()]

In [192]:
outs = [mdl(torch.tensor([i])[None, None, -1].cuda()) for i in idxs]

In [254]:
mdl2.cuda()

BasicLanguageModel2(
  (i_h): Embedding(3096, 64)
  (h_h): Linear(in_features=64, out_features=64, bias=True)
  (h_o): Linear(in_features=64, out_features=3096, bias=True)
  (bn): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
)

In [250]:
mdl

BasicLanguageModel(
  (i_h): Embedding(3096, 64)
  (h_h): Linear(in_features=64, out_features=64, bias=True)
  (h_o): Linear(in_features=64, out_features=3096, bias=True)
  (bn): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
)

In [248]:
mdl2(torch.tensor([[301]]).cuda())

RuntimeError: Expected object of backend CPU but got backend CUDA for argument #3 'index'

In [266]:
mdl2(torch.tensor([[[301]]]).cuda())

RuntimeError: running_mean should contain 1 elements not 64

In [194]:
outs[0].size()

torch.Size([1, 1, 64, 3096])

In [195]:
outs[-1].size()

torch.Size([1, 1, 64, 3096])

In [196]:
len(outs)

3

In [229]:
outs[1][0][0][63].size()

torch.Size([3096])

In [319]:
mdl = learn.model

In [323]:
outs = mdl(torch.tensor([[301]]).cuda())

In [327]:
outs[2].size()

torch.Size([1, 3096])

In [328]:
mdl

BasicLanguageModel(
  (i_h): Embedding(3096, 64)
  (h_h): Linear(in_features=64, out_features=64, bias=True)
  (h_o): Linear(in_features=64, out_features=3096, bias=True)
  (bn): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
)